In [1]:
pip install langchain langchain-groq chromadb

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key = 'YOUR_API_KEY',
    model="llama-3.1-70b-versatile",
)

response = llm.invoke("what do human like most cats or dogs, just give me one word answer")
print(response.content)

Dogs.


In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.accenture.com/in-en/careers/jobdetails?id=ATCI-4597357-S1779506_en&title=Application%20Developer")
page_data = loader.load().pop().page_content


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
res.content

'[\n  {\n    "role": "Application Developer",\n    "experience": "Minimum 3 years",\n    "skills": "Veeva CRM, Veeva Vault",\n    "description": "Design, build and configure applications to meet business process and application requirements. Must have skills: Veeva CRM. Good to have skills: Veeva Vault. Minimum 3 year(s) of experience is required. Educational Qualification: 15 years full time education."\n  }\n]'

In [5]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Application Developer',
  'experience': 'Minimum 3 years',
  'skills': 'Veeva CRM, Veeva Vault',
  'description': 'Design, build and configure applications to meet business process and application requirements. Must have skills: Veeva CRM. Good to have skills: Veeva Vault. Minimum 3 year(s) of experience is required. Educational Qualification: 15 years full time education.'}]

In [6]:
import pandas as pd

df = pd.read_csv("skiill and resume.csv")
df

,Techstack,Links
0,Java,file:///C:/Users/LIKHITH%20GOWDA%20K/Downloads...
1,Javascript,file:///C:/Users/LIKHITH%20GOWDA%20K/Downloads...
2,Python,file:///C:/Users/LIKHITH%20GOWDA%20K/Downloads...
3,NodeJs,file:///C:/Users/LIKHITH%20GOWDA%20K/Downloads...
4,"HTML , HTML5",file:///C:/Users/LIKHITH%20GOWDA%20K/Downloads...
5,Css,file:///C:/Users/LIKHITH%20GOWDA%20K/Downloads...
6,Mysql,file:///C:/Users/LIKHITH%20GOWDA%20K/Downloads...
7,"Machine Learning, TensorFlow",file:///C:/Users/LIKHITH%20GOWDA%20K/Downloads...
8,"Full-stack, Express.js",file:///C:/Users/LIKHITH%20GOWDA%20K/Downloads...
9,Backend,file:///C:/Users/LIKHITH%20GOWDA%20K/Downloads...


In [7]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])
        


In [8]:
job = {
    'skills': ['Python', 'Machine Learning']  # Define with actual skills you are querying for
}

links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'file:///C:/Users/LIKHITH%20GOWDA%20K/Downloads/Likhith-gowda-K.pdf'},
  {'links': 'file:///C:/Users/LIKHITH%20GOWDA%20K/Downloads/Likhith-gowda-K.pdf'}],
 [{'links': 'file:///C:/Users/LIKHITH%20GOWDA%20K/Downloads/Likhith-gowda-K.pdf'},
  {'links': 'file:///C:/Users/LIKHITH%20GOWDA%20K/Downloads/Likhith-gowda-K.pdf'}]]

In [9]:

job = json_res[0]  # Access the first item in the list
job_skills = job['skills']  # Now this should work
print(job_skills)


Veeva CRM, Veeva Vault


In [10]:
prompt_email = PromptTemplate.from_template(
       """
### JOB DESCRIPTION:
{job_description}

### INSTRUCTION:
You are an experienced professional currently seeking new opportunities. In the past, you have developed expertise in fields relevant to the job description, contributing to notable achievements in process optimization, cost reduction, and efficiency enhancements in previous roles. 
Your task is to write a personalized cover letter addressed to the hiring manager of the company. Emphasize your relevant skills, experience, and enthusiasm for the position described above. Highlight any past projects or skills that align closely with the responsibilities mentioned. Use the following links as examples to showcase similar past work or demonstrate your skills effectively: {link_list}
Keep your tone professional, enthusiastic, and goal-oriented. Do not provide a preamble.
### COVER LETTER (NO PREAMBLE):
"""
)

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Dear Hiring Manager,

I am excited to apply for the Application Developer position at your esteemed organization, as advertised. With over 3 years of experience in designing, building, and configuring applications to meet business process and application requirements, I am confident that my skills and expertise make me an ideal candidate for this role.

As a seasoned professional with a strong background in Veeva CRM, I have successfully developed and implemented numerous applications that have resulted in significant process optimization, cost reduction, and efficiency enhancements in my previous roles. My expertise in Veeva CRM has enabled me to streamline business processes, improve data management, and enhance user experience. I am excited about the prospect of leveraging my skills to drive similar success in your organization.

In addition to my Veeva CRM expertise, I also possess knowledge of Veeva Vault, which I believe would be a valuable asset in this role. My experience with 